# HeadHunt

We're going to teach a program what kinds of jobs we like, using labeled Gmail messages as a training source.

Suggested fields:
timestamp type source raw text label

# Where was I?

In [ ]:
import pickle 
import sys

MATCHWELL = '../matchwell'
if MATCHWELL not in sys.path:
    sys.path.insert(0, MATCHWELL)

import pandas as pd
%load_ext autoreload
%autoreload 2
from matchwell import models, gmail

# Constants
POS_LABEL = 'job-offers/yes'
NEG_LABEL = 'job-offers-no'
DATAFILE = './dataframe.h5'
CLASSIFIER = './text_pipeline.pkl'

In [ ]:
# Load DataFrame
with open(DATAFILE) as f:
    df = pd.read_hdf(DATAFILE) 

In [ ]:
# Load classifier
clf = models.load_model(CLASSIFIER)

gml = gmail.Gmail().connect()

In [ ]:
!ls -lh